# Загрузка "потока событий" в Redis

In [2]:
from typing import Dict, List

import redis
import json

import pandas as pd


class Constants:
    USER_ID = "user_id"
    ITEM_ID = "item_id"
    ITEMS_IDS = "items_ids"
    TIMESTAMP = "time"

    TRANSACTIONS_PATH = "/Users/alfa/Documents/diplom/graphnn-recommendation-system/data/processed_transactions_train.csv"


def load_data_to_redis(user_sequences: List[Dict], r: redis.Redis) -> None:
    """
    Загрузка данных в Redis
    """
    for user_sequence in user_sequences:
        user_id = user_sequence.get(Constants.USER_ID)
        items_ids = user_sequence.get(Constants.ITEMS_IDS)

        r.set(f"user:{user_id}", json.dumps(items_ids))

    print("Данные успешно загружены в Redis")


# Подключение к Redis
r = redis.Redis(
    host="localhost",
    password="redis123",
    port=6379,
    decode_responses=True,
)

In [3]:
# Загрузка предобработанных транзакций
transactions = pd.read_parquet(Constants.TRANSACTIONS_PATH)

# Подготовка последовательностей для каждого пользователя
user_sequences = (
    transactions.groupby([Constants.USER_ID], as_index=False)[Constants.ITEM_ID]
    .apply(list)
    .rename(columns={Constants.ITEM_ID: Constants.ITEMS_IDS})
    .to_dict("records")
)

# Загрузка данных в Redis
load_data_to_redis(user_sequences=user_sequences, r=r)

Данные успешно загружены в Redis


In [8]:
# Проверка данных для случайного пользователя
user_id = "02703e242da346fc85f1aad21208d1103e62e705564c81ca2b6edf943032320a"
data = r.get(f"user:{user_id}")

if data:
    items = json.loads(data)  # Десериализация из JSON
    print(f"Товары пользователя {user_id}: {items}")
else:
    print("Ключ не найден")

Товары пользователя 02703e242da346fc85f1aad21208d1103e62e705564c81ca2b6edf943032320a: [573716012, 573716012, 568597006, 568597006, 673677002, 673677002, 673677002, 565379002, 685816002, 562245050, 562245050, 399256005, 399256005, 573716012, 573716012, 562245050, 573716012, 573716012, 565379001, 730683001, 158340001, 111593001, 568601006, 568601006, 573716012, 573716012, 568597006, 573716012, 573716012, 573716012, 573716012, 573716012, 573716012, 573716012, 573716012, 573716012, 507909001, 599580038, 536139006, 684209013, 684209013, 572797002, 572797002, 572797002, 572797001, 572797001]


In [ ]:
import requests

response = requests.get(
    "http://localhost:8000/recommendations/personal_items/",
    params={
        "user_id": "test"
    },
)
response.json()